This notebook tries to keep things generalised staring with feature engineering we genrate basic features like:


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/spooky-author-identification/test.csv
/kaggle/input/spooky-author-identification/train.csv
/kaggle/input/spooky-author-identification/sample_submission.csv
/kaggle/input/spooky-author-identification/test/test.csv
/kaggle/input/spooky-author-identification/sample_submission/sample_submission.csv
/kaggle/input/spooky-author-identification/train/train.csv


In [2]:
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/spooky-author-identification/test.csv
/kaggle/input/spooky-author-identification/train.csv
/kaggle/input/spooky-author-identification/sample_submission.csv
/kaggle/input/spooky-author-identification/test/test.csv
/kaggle/input/spooky-author-identification/sample_submission/sample_submission.csv
/kaggle/input/spooky-author-identification/train/train.csv


In [4]:
test=pd.read_csv("/kaggle/input/spooky-author-identification/test.csv")
train=pd.read_csv("/kaggle/input/spooky-author-identification/train.csv")
sample=pd.read_csv("/kaggle/input/spooky-author-identification/sample_submission.csv")

In [5]:
sample.head()

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698


In [6]:
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [7]:
train.shape

(19579, 3)

In [8]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [9]:
train.shape

(19579, 3)

In [10]:
train['author'].value_counts()

EAP    7900
MWS    6044
HPL    5635
Name: author, dtype: int64

In [11]:
#getting the length of the text and storing it as feature
train['num_feature']=train['text'].apply(len)

In [12]:
#checking the new column
train.head()

,id,text,author,num_feature
0,id26305,"This process, however, afforded me no means of...",EAP,231
1,id17569,It never once occurred to me that the fumbling...,HPL,71
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,200
3,id27763,How lovely is spring As we looked from Windsor...,MWS,206
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,174


In [13]:
def word_count(string):
    words=string.split()
    return len(words)

In [14]:
train['num_words']=train['text'].apply(word_count)
train.head()

,id,text,author,num_feature,num_words
0,id26305,"This process, however, afforded me no means of...",EAP,231,41
1,id17569,It never once occurred to me that the fumbling...,HPL,71,14
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,200,36
3,id27763,How lovely is spring As we looked from Windsor...,MWS,206,34
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,174,27


In [15]:
def remove_punctuation(Text):
    translator = str.maketrans('', '', string.punctuation)
    return Text.translate(translator)

In [16]:
train['text'] = train['text'].apply(remove_punctuation)

In [17]:
train.head()

,id,text,author,num_feature,num_words
0,id26305,This process however afforded me no means of a...,EAP,231,41
1,id17569,It never once occurred to me that the fumbling...,HPL,71,14
2,id11008,In his left hand was a gold snuff box from whi...,EAP,200,36
3,id27763,How lovely is spring As we looked from Windsor...,MWS,206,34
4,id12958,Finding nothing else not even gold the Superin...,HPL,174,27


In [18]:
sw = stopwords.words('english')
# displaying the stopwords
np.array(sw)

array(['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you',
       "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself',
       'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her',
       'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them',
       'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom',
       'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are',
       'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',
       'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and',
       'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at',
       'by', 'for', 'with', 'about', 'against', 'between', 'into',
       'through', 'during', 'before', 'after', 'above', 'below', 'to',
       'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under',
       'again', 'further', 'then', 'once', 'here', 'there', 'when',
       'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'm

In [19]:
def stopwords1(text):
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    return " ".join(text)

In [20]:
train['text'] = train['text'].apply(stopwords1)

In [21]:
train.head()

,id,text,author,num_feature,num_words
0,id26305,process however afforded means ascertaining di...,EAP,231,41
1,id17569,never occurred fumbling might mere mistake,HPL,71,14
2,id11008,left hand gold snuff box capered hill cutting ...,EAP,200,36
3,id27763,lovely spring looked windsor terrace sixteen f...,MWS,206,34
4,id12958,finding nothing else even gold superintendent ...,HPL,174,27


In [22]:
stemmer = SnowballStemmer("english")
def stemming(text):    
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text)

In [23]:
train['text'] = train['text'].apply(stemming)

In [24]:
train.head()

,id,text,author,num_feature,num_words
0,id26305,process howev afford mean ascertain dimens dun...,EAP,231,41
1,id17569,never occur fumbl might mere mistak,HPL,71,14
2,id11008,left hand gold snuff box caper hill cut manner...,EAP,200,36
3,id27763,love spring look windsor terrac sixteen fertil...,MWS,206,34
4,id12958,find noth els even gold superintend abandon at...,HPL,174,27


In [25]:
from sklearn import preprocessing
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

v = TfidfVectorizer()
x = v.fit_transform(train['text'])

In [27]:
df1 = pd.DataFrame(x.toarray(), columns=v.get_feature_names())
print(df1)

       aaem   ab  aback  abaft   abandon  abaout  abas  abash  abat  abbey  \
0       0.0  0.0    0.0    0.0  0.000000     0.0   0.0    0.0   0.0    0.0   
1       0.0  0.0    0.0    0.0  0.000000     0.0   0.0    0.0   0.0    0.0   
2       0.0  0.0    0.0    0.0  0.000000     0.0   0.0    0.0   0.0    0.0   
3       0.0  0.0    0.0    0.0  0.000000     0.0   0.0    0.0   0.0    0.0   
4       0.0  0.0    0.0    0.0  0.266318     0.0   0.0    0.0   0.0    0.0   
...     ...  ...    ...    ...       ...     ...   ...    ...   ...    ...   
19574   0.0  0.0    0.0    0.0  0.000000     0.0   0.0    0.0   0.0    0.0   
19575   0.0  0.0    0.0    0.0  0.000000     0.0   0.0    0.0   0.0    0.0   
19576   0.0  0.0    0.0    0.0  0.000000     0.0   0.0    0.0   0.0    0.0   
19577   0.0  0.0    0.0    0.0  0.000000     0.0   0.0    0.0   0.0    0.0   
19578   0.0  0.0    0.0    0.0  0.000000     0.0   0.0    0.0   0.0    0.0   

       ...  æmilianus  æneid  ærial  æronaut  ærostat  æschylus

In [28]:
res = pd.concat([train, df1], axis=1)
print(res)

            id                                               text author  \
0      id26305  process howev afford mean ascertain dimens dun...    EAP   
1      id17569                never occur fumbl might mere mistak    HPL   
2      id11008  left hand gold snuff box caper hill cut manner...    EAP   
3      id27763  love spring look windsor terrac sixteen fertil...    MWS   
4      id12958  find noth els even gold superintend abandon at...    HPL   
...        ...                                                ...    ...   
19574  id17718  could fanci look emin landscap painter built b...    EAP   
19575  id08973                            lid clench togeth spasm    EAP   
19576  id05267  mai il faut agir say frenchman never faint out...    EAP   
19577  id17513             item news like strike us coolli receiv    EAP   
19578  id00393  laid gnarl claw shoulder seem shake altogeth m...    HPL   

       num_feature  num_words  aaem   ab  aback  abaft   abandon  ...  \
0             

In [29]:
res.shape

(19579, 15147)

In [30]:
res=res.drop(['text'], axis=1)
res=res.drop(['id'], axis=1)
#res=res.drop(['author'], axis=1)
res=res.drop(['author'], axis=1)
#res.head()
#res_encoded = pd.get_dummies(res)

In [31]:
res.dtypes

num_feature      int64
num_words        int64
aaem           float64
ab             float64
aback          float64
                ...   
æschylus       float64
élite          float64
émeut          float64
οἶδα           float64
υπνος          float64
Length: 15141, dtype: object

In [32]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(res, y,
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [33]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [34]:
y.shape

(19579,)

In [35]:
res.shape

(19579, 15141)

In [36]:
#naive bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)
predictions = clf.predict_proba(X_test)
print(predictions)
#print(predictions.shape)
from sklearn.metrics import log_loss
a=log_loss(y_test, predictions)
print("The log loss score is:",a)

[[0.77531679 0.069666   0.15501721]
 [0.68104774 0.14385774 0.17509452]
 [0.7308182  0.04274451 0.22643729]
 ...
 [0.20883415 0.1626255  0.62854035]
 [0.25617599 0.1332726  0.61055142]
 [0.28943624 0.58795937 0.12260439]]
The log loss score is: 0.6045560344976592


In [37]:
#logistic regression
from sklearn.linear_model import LogisticRegression
reg=LogisticRegression(C=1.0)
reg.fit(X_train, y_train)
predictions = reg.predict_proba(X_test)
print(predictions)
#print(predictions.shape)
from sklearn.metrics import log_loss
a=log_loss(y_test, predictions)
print("The log loss score is:",a)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[[0.45355738 0.12555015 0.42089247]
 [0.78230287 0.13429561 0.08340153]
 [0.64270546 0.06756325 0.28973129]
 ...
 [0.19248714 0.15269916 0.65481369]
 [0.2863129  0.17228902 0.54139808]
 [0.06968167 0.8291733  0.10114503]]
The log loss score is: 0.5925019666420605


In [38]:
#xgboost
import xgboost as xgb
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(X_train, y_train)
predictions = clf.predict_proba(X_test)
print(predictions)
#print(predictions.shape)
from sklearn.metrics import log_loss
a=log_loss(y_test, predictions)
print("The log loss score is:",a)

[[0.36051592 0.21142825 0.42805585]
 [0.66596633 0.15471594 0.17931773]
 [0.43869507 0.27964336 0.2816616 ]
 ...
 [0.39515486 0.21163663 0.39320856]
 [0.43292087 0.23490855 0.3321706 ]
 [0.21123092 0.6566484  0.13212068]]
The log loss score is: 0.7344408892439935


In [39]:
#losgistic regression with hyperparameter tuning
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1, 1.0, 10], 'penalty': ['l1', 'l2']}
reg=LogisticRegression()
reg_cv=GridSearchCV(reg,param_grid,cv=4)
reg_cv.fit(X_train, y_train)
predictions = reg_cv.predict_proba(X_test)
print(predictions)
#print(predictions.shape)
b=reg_cv.best_score_
c=reg_cv.best_params_
print("best score",b)
from sklearn.metrics import log_loss
a=log_loss(y_test, predictions)
print("The log loss is:",a)
print("Best values found via grid search",c)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase

[[0.4519483  0.07689424 0.47115746]
 [0.9578433  0.02686628 0.01529042]
 [0.61653932 0.01278344 0.37067724]
 ...
 [0.06708989 0.18712637 0.74578374]
 [0.0304077  0.10632155 0.86327075]
 [0.0195813  0.96493602 0.01548268]]
best score 0.8103966857726577
The log loss is: 0.47713966276024544
Best values found via grid search {'C': 10, 'penalty': 'l2'}


In [40]:
#naive bazyes with hyperparameter tuning
clf = MultinomialNB()
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
clf_cv=GridSearchCV(clf,param_grid,cv=5)
clf_cv.fit(X_train, y_train)
predictions = clf_cv.predict_proba(X_test)
print(predictions)
#print(predictions.shape)
b=clf_cv.best_score_
c=clf_cv.best_params_
print("best score",b)
#from sklearn.metrics import log_loss
a=log_loss(y_test, predictions)
print("The log loss is:",a)
print("Best values found via grid search",c)

[[0.93159119 0.03292072 0.03548809]
 [0.7159039  0.10740301 0.17669309]
 [0.61940406 0.05498789 0.32560805]
 ...
 [0.12296932 0.16188363 0.71514705]
 [0.0614737  0.10954891 0.82897739]
 [0.02834984 0.95798414 0.01366602]]
best score 0.8349696384995177
The log loss is: 0.4616788906041756
Best values found via grid search {'alpha': 0.1}
